In [2]:
import numpy as np
import pandas as pd
import pickle
from keras.datasets import cifar10

2023-06-14 05:20:37.577534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 05:20:37.956632: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 05:20:37.958759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 05:20:39.425367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Load training and testing data separately
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


170498071/170498071 [==============================] - 46s 0us/step


In [5]:
# Reshape x_train from 4D to 2D array (number of samples, width*height*channels)
x_train = x_train.reshape(x_train.shape[0], -1)

# Reshape y_train to 1D array
y_train = y_train.reshape(-1)

# Combine training data and labels into a single numpy array for easier manipulation
train_data = np.column_stack((x_train, y_train))

# Randomly shuffle the training data
np.random.shuffle(train_data)


In [63]:

# Number of chunks
n_chunks = 10

# Generate sizes following a power-law distribution
sizes = np.random.zipf(1.5, n_chunks)

# Normalize the sizes so that their sum equals the number of training samples
sizes = (sizes / sizes.sum() * len(train_data)).astype(int)

# Ensure that the sum of sizes is equal to the total number of training samples
sizes[-1] += len(train_data) - sizes.sum()


sizes

array([ 2173,  2173,  8695,  2173,  4347,  2173, 13043,  2173, 10869,
        2181])

In [42]:
# # Number of chunks
# n_chunks = 10

# # Generate sizes following a power-law distribution
# sizes = np.random.zipf(1.5, n_chunks)

# # Normalize the sizes so that their sum equals the number of training samples
# sizes = (sizes / sizes.sum() * len(x_train)).astype(int)

# # Ensure that the sum of sizes is equal to the total number of training samples
# sizes[-1] += len(x_train) - sizes.sum()

# sizes

array([2631, 7894, 2631, 7894, 5263, 2631, 5263, 7894, 5263, 2636])

array([1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501])

In [64]:
# Create an array of indices at which to split the training data
split_indices = np.cumsum(sizes)[:-1]

# Split the training data into chunks of different sizes
chunks = np.split(train_data, split_indices)


In [65]:

# Function to get label distribution in a chunk
def get_label_distribution(chunk):
    # The label is in the last column
    labels = chunk[:, -1]
    unique_labels, counts = np.unique(labels, return_counts=True)
    return dict(zip(unique_labels, counts))

# Store size and label distribution of each chunk, and save each chunk as a pickle file
chunk_info = []
for i, chunk in enumerate(chunks):
    info = {}
    info['chunk'] = i+1
    info['size'] = len(chunk)
    info['label_distribution'] = get_label_distribution(chunk)
    chunk_info.append(info)

    # Save chunk as a pickle file
    with open(f'../data/cifar_dataset/chunks/chunk_{i+1}.pickle', 'wb') as f:
        pickle.dump(chunk, f)



# Save test set as a pickle file
with open('../data/cifar_dataset/chunks/test_set.pickle', 'wb') as f:
    pickle.dump((x_test, y_test), f)

In [66]:
# Convert list of dictionaries to DataFrame for better visualization
df = pd.DataFrame(chunk_info)

# print dataframe
print(df)

   chunk   size                                 label_distribution
0      1   2173  {0: 211, 1: 226, 2: 215, 3: 207, 4: 232, 5: 21...
1      2   2173  {0: 209, 1: 212, 2: 228, 3: 230, 4: 223, 5: 24...
2      3   8695  {0: 867, 1: 915, 2: 852, 3: 840, 4: 858, 5: 87...
3      4   2173  {0: 203, 1: 201, 2: 211, 3: 242, 4: 230, 5: 20...
4      5   4347  {0: 447, 1: 425, 2: 397, 3: 417, 4: 462, 5: 45...
5      6   2173  {0: 216, 1: 210, 2: 226, 3: 224, 4: 209, 5: 19...
6      7  13043  {0: 1231, 1: 1256, 2: 1333, 3: 1319, 4: 1328, ...
7      8   2173  {0: 228, 1: 240, 2: 221, 3: 188, 4: 210, 5: 19...
8      9  10869  {0: 1166, 1: 1086, 2: 1085, 3: 1120, 4: 1056, ...
9     10   2181  {0: 222, 1: 229, 2: 232, 3: 213, 4: 192, 5: 24...


In [67]:
# Save dataframe to csv
df.to_csv("../data/cifar_dataset/chunks_info.csv", index=False)


In [68]:
df.size

30

In [69]:
df

,chunk,size,label_distribution
0,1,2173,"{0: 211, 1: 226, 2: 215, 3: 207, 4: 232, 5: 21..."
1,2,2173,"{0: 209, 1: 212, 2: 228, 3: 230, 4: 223, 5: 24..."
2,3,8695,"{0: 867, 1: 915, 2: 852, 3: 840, 4: 858, 5: 87..."
3,4,2173,"{0: 203, 1: 201, 2: 211, 3: 242, 4: 230, 5: 20..."
4,5,4347,"{0: 447, 1: 425, 2: 397, 3: 417, 4: 462, 5: 45..."
5,6,2173,"{0: 216, 1: 210, 2: 226, 3: 224, 4: 209, 5: 19..."
6,7,13043,"{0: 1231, 1: 1256, 2: 1333, 3: 1319, 4: 1328, ..."
7,8,2173,"{0: 228, 1: 240, 2: 221, 3: 188, 4: 210, 5: 19..."
8,9,10869,"{0: 1166, 1: 1086, 2: 1085, 3: 1120, 4: 1056, ..."
9,10,2181,"{0: 222, 1: 229, 2: 232, 3: 213, 4: 192, 5: 24..."


In [71]:
df.label_distribution


0    {0: 211, 1: 226, 2: 215, 3: 207, 4: 232, 5: 21...
1    {0: 209, 1: 212, 2: 228, 3: 230, 4: 223, 5: 24...
2    {0: 867, 1: 915, 2: 852, 3: 840, 4: 858, 5: 87...
3    {0: 203, 1: 201, 2: 211, 3: 242, 4: 230, 5: 20...
4    {0: 447, 1: 425, 2: 397, 3: 417, 4: 462, 5: 45...
5    {0: 216, 1: 210, 2: 226, 3: 224, 4: 209, 5: 19...
6    {0: 1231, 1: 1256, 2: 1333, 3: 1319, 4: 1328, ...
7    {0: 228, 1: 240, 2: 221, 3: 188, 4: 210, 5: 19...
8    {0: 1166, 1: 1086, 2: 1085, 3: 1120, 4: 1056, ...
9    {0: 222, 1: 229, 2: 232, 3: 213, 4: 192, 5: 24...
Name: label_distribution, dtype: object